# Insiders Analysis

Transaction Codes

P: Open market or private purchase of non-derivative or derivative security

M: Exercise or conversion of derivative security exempted puruant to Rule 16b-3. Indicates
   that an insider has exercised options, typically converting them into shares. Can be seen as a form
   of indirect buying, though its a bit more nuanced because the insider may immediately sell the shares 
   they've acquired

S: Indicates an open market or private sale of a non derivative or derivative security.

In [9]:

import pandas as pd
import nasdaqdatalink

import numpy as np
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
NASDAQ_KEY = "zxLNKQydu_qNXQ2tZ7vz"

nasdaqdatalink.ApiConfig.api_key = NASDAQ_KEY


## Requesting Insider Data 

In [10]:
insiders = nasdaqdatalink.get_table('SHARADAR/SF2', ticker='CHTR')
insiders_df = pd.DataFrame(insiders, columns=["ticker", 'filingdate', 'transactiondate', 'formtype', 'issuername', 'ownername', 'officertitle', 'istenpercentowner', 'transactioncode', 'transactionpricepershare', 'transactionvalue', 'securitytitle'])
insiders_df = insiders_df.sort_values(by="filingdate", ascending=False)
insiders_df

,ticker,filingdate,transactiondate,formtype,issuername,ownername,officertitle,istenpercentowner,transactioncode,transactionpricepershare,transactionvalue,securitytitle
None,,,,,,,,,,,,
38,CHTR,2024-01-17,2024-01-12,4,CHARTER COMMUNICATIONS IN,WINFREY CHRISTOPHER L,President and CEO,N,F,366.88,179771.0,Class A Common Stock
42,CHTR,2024-01-17,2024-01-12,4,CHARTER COMMUNICATIONS IN,WINFREY CHRISTOPHER L,President and CEO,N,F,364.90,7527887.0,Class A Common Stock
1977,CHTR,2024-01-17,2024-01-12,4,CHARTER COMMUNICATIONS IN,FISCHER JESSICA M,Chief Financial Officer,N,M,NaN,NaN,Class A Common Stock
31,CHTR,2024-01-17,2024-01-12,4,CHARTER COMMUNICATIONS IN,WINFREY CHRISTOPHER L,President and CEO,N,M,NaN,NaN,Restricted Stock Units
32,CHTR,2024-01-17,2024-01-12,4,CHARTER COMMUNICATIONS IN,WINFREY CHRISTOPHER L,President and CEO,N,M,150.88,1216847.0,Stock Options
...,...,...,...,...,...,...,...,...,...,...,...,...
578,CHTR,2010-09-14,NaT,3,CHARTER COMMUNICATIONS IN,OAKTREE CAPITAL GROUP HOLDINGS GP LLC,None,Y,None,NaN,NaN,Class A Common Stock
2564,CHTR,2010-09-14,NaT,3,CHARTER COMMUNICATIONS IN,ALLEN PAUL G,None,N,None,NaN,NaN,Class A Common Stock
490,CHTR,2010-09-14,NaT,3,CHARTER COMMUNICATIONS IN,RIGDON GREG S,EVPCorp Devel Strategy,N,None,NaN,NaN,Class A Common Stock


## Screening for clustered Insider buying

Here we'll get data in the most recent 3 months and screen for insider buying

In [12]:
universe_path = './active_equities.csv'
universe_df = pd.read_csv(universe_path)
symbols = universe_df['ticker']


# Get three month ago date
today = datetime.today()
three_months_ago = today - timedelta(days=90)
formatted_date = three_months_ago.strftime('%Y-%m-%d')

transaction_code_map = {
    "P": "Purchase",
    "M": "Exercise of Derivative",
    "S": "Sale"
}

cluster_buying_companies = []

for symbol in symbols:
    try:
        insiders = nasdaqdatalink.get_table('SHARADAR/SF2', filingdate={'gte': formatted_date}, ticker=symbol)
        insiders_df = pd.DataFrame(insiders, columns=["ticker", 'filingdate', 'transactiondate','issuername', 'ownername', 'officertitle','transactioncode', 'transactionpricepershare', 'transactionvalue'])
        insiders_df = insiders_df.sort_values(by="filingdate", ascending=False)
        # Check for any insider purchases in the last 3 months
        insider_buys_df = insiders_df[insiders_df['transactioncode'] == 'P']

        # Here we'll determine cluster insider buying to be 3 or more insiders
        if len(insider_buys_df) > 2:
            clustered_transaction_sum = insider_buys_df['transactionvalue'].sum()
            if clustered_transaction_sum > 1000000:
                row_values = {
                    'ticker': symbol,
                    'companyName': insiders_df['issuername'][0],
                    'numOfInsiders': len(insider_buys_df),
                    'transactionTotal': clustered_transaction_sum
                }
                cluster_buying_companies.append(row_values)
    
    except Exception as e:
        print(f"Error getting insider data: {e}")
        break

    

clustered_buying_df = pd.DataFrame(cluster_buying_companies)
clustered_buying_df.to_csv('clusterBuying.csv', index=False)



KeyboardInterrupt: 